**Notebook utilizado para um estudo sobre árvore de decisão e floresta randômica**


# **Executando o Pyspark no Colab**

Para executar o spark no Colab, precisamos primeiro instalar todas as dependências no ambiente Colab, ou seja, Apache Spark 2.3.2 com hadoop 2.7, Java 8 e Findspark para localizar o spark no sistema. Siga as etapas para instalar as dependências:

In [65]:
!sudo apt update
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [66]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ArvoreDecisao") \
    .getOrCreate()
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.types import *
from pyspark.sql.functions import *

Agora com o Colab pronto para executar o Spark, será construido um modelo de Árvore de Decisão.

# **Árvore de Decisão**

A árvore de decisão é um dos métodos mais comuns no aprendizado de máquina mais antigas e amplamente utilizadas, que pressupõe uma relação entre variáveis. Esses algoritmos subdividem progressivamente os dados em conjuntos cada vez menores e mais específicos, em termos de seus atributos, até atingirem um tamanho simplificado o bastante para ser rotulado. Para isso é necessário treinar o modelo com dados previamente rotulados, de modo a aplicá-lo a dados novos.

O objetivo deste exercício é prever a espécie de íris de uma flor pelas características dadas. Vamos prever a espécie através do conjunto de dados de  3 espécies de íris onde foram coletadas 50 amostras para cada uma espécie, considerando a espécie como a variável de saída e as outras variáveis de tamanhos de pétalas e sépalas como entrada.

Baixe o conjunto de dados [aqui](https://www.kaggle.com/uciml/iris) e mantenha-o em algum lugar no seu computador. Carregue o conjunto de dados em seu diretório Colab a partir de seu sistema local:

## **Conhecendo o Bando de Dados**

In [67]:
#Diretório que contém o arquivo a ser utilizado
diretorioArvore="iris_bezdekIris.csv"  

Inicialmente é necessário carregar o conjunto de dados, após isso, é possivel definir um esquema para uma analise prévia dos mesmos

In [68]:
#Lendo arquivos armazenados CSV com o esquema definido
df_iris = spark.read.format('csv').options(inferSchema=True,header='false',delimiter=',').load(diretorioArvore)

Observe que usamos InferSchema dentro do read.csv. InferSchema nos permite inferir automaticamente diferentes tipos de dados para cada coluna.

Vamos imprimir uma olhada no conjunto de dados para ver os tipos de dados de cada coluna:

In [69]:
df_iris.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)



In [70]:
df_iris.show(5)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 5 rows



A fim de gerar melhor entendimento, os nomes das colunas no cabeçalho vão ser alterados, permitindo assim identificar com maior clareza a qual dado aquela informação corresponde.

In [71]:
#Modificando o nome das colunas existentes no cabeçalho 
df_iris = df_iris.selectExpr("_c0 as sep_len", "_c1 as sep_wid", "_c2 as pet_len", "_c3 as pet_wid", "_c4 as label")

In [72]:
df_iris.show(5)

+-------+-------+-------+-------+-----------+
|sep_len|sep_wid|pet_len|pet_wid|      label|
+-------+-------+-------+-------+-----------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|
+-------+-------+-------+-------+-----------+
only showing top 5 rows



Por fim, uma analise prévia das informações apresentadas no conjunto de dados

In [73]:
#Encontrando as "estatísticas"
df_iris.describe(['sep_len','sep_wid','pet_len','pet_wid']).show()

+-------+------------------+-------------------+------------------+------------------+
|summary|           sep_len|            sep_wid|           pet_len|           pet_wid|
+-------+------------------+-------------------+------------------+------------------+
|  count|               150|                150|               150|               150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|
|    min|               4.3|                2.0|               1.0|               0.1|
|    max|               7.9|                4.4|               6.9|               2.5|
+-------+------------------+-------------------+------------------+------------------+



In [74]:
#Definindo a visão do dataframe para ser utilizado como uma tabela pelo SQL
df_iris.createOrReplaceTempView("irisTable")
display(spark.sql('select * from irisTable'))

DataFrame[sep_len: double, sep_wid: double, pet_len: double, pet_wid: double, label: string]


## **Iniciando o Processo de Construção e Aplicação da Árvore de Decisão**

Agora que carregamos o conjunto de dados, podemos começar a analisar.
Para a árvore de decisão, precisamos importar dois módulos do Pyspark, ou seja, Vector Assembler e StringIndexer. O Vector Assembler é um transformador que reúne todos os recursos em um vetor a partir de várias colunas que contêm o tipo duplo. O StringIndexer é utilizado para as colunas quem contém valores de string para convertê-la em valores numéricos.

In [75]:
from pyspark.ml.linalg import Vectors  #Biblioteca que contém funções de Algebra Linear
from pyspark.ml.feature import VectorAssembler #Biblioteca que contém as funções para a construção de vetores
from pyspark.ml.feature import StringIndexer  #Cria o 'vetor' para cada uma das classes existentes na coluna label

O próximo passo é converter todos os recursos de diferentes colunas em uma única coluna e vamos chamar esta nova coluna de vetor como 'features' no outputCol

In [76]:
#Criando o vetor de características
vector_assembler = VectorAssembler(inputCols=["sep_len", "sep_wid", "pet_len", "pet_wid"],outputCol="features")
df_temp = vector_assembler.transform(df_iris)
df_temp.show(5)

+-------+-------+-------+-------+-----------+-----------------+
|sep_len|sep_wid|pet_len|pet_wid|      label|         features|
+-------+-------+-------+-------+-----------+-----------------+
|    5.1|    3.5|    1.4|    0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|    4.9|    3.0|    1.4|    0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|    4.7|    3.2|    1.3|    0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|    4.6|    3.1|    1.5|    0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|    5.0|    3.6|    1.4|    0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-------+-------+-------+-------+-----------+-----------------+
only showing top 5 rows



In [77]:
#Removendo as colunas que não serão utilizadas
df_menor = df_temp.drop('sep_len', 'sep_wid', 'pet_len', 'pet_wid')
df_menor.show(5)

+-----------+-----------------+
|      label|         features|
+-----------+-----------------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|
|Iris-setosa|[4.9,3.0,1.4,0.2]|
|Iris-setosa|[4.7,3.2,1.3,0.2]|
|Iris-setosa|[4.6,3.1,1.5,0.2]|
|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+-----------------+
only showing top 5 rows



A coluna 'features' refere-se aos recursos de entrada de todas as colunas e 'label' é a coluna de destino.

In [78]:
#Aplicando as transformações para a coluna label
l_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")  #Cria o objeto para a codificação
df_final = l_indexer.fit(df_menor).transform(df_menor)  #Aplica a transformação

In [79]:
df_final.show(5)

+-----------+-----------------+----------+
|      label|         features|labelIndex|
+-----------+-----------------+----------+
|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|Iris-setosa|[5.0,3.6,1.4,0.2]|       0.0|
+-----------+-----------------+----------+
only showing top 5 rows



Em seguida, devemos dividir os dados de treinamento e teste de acordo com nosso conjunto de dados (0,7 e 0,3 neste caso).

In [80]:
#Dividindo entre dados de treinamento e teste
(train, test) = df_final.randomSplit([0.7, 0.3])

In [81]:
test.show(5)

+-----------+-----------------+----------+
|      label|         features|labelIndex|
+-----------+-----------------+----------+
|Iris-setosa|[4.4,2.9,1.4,0.2]|       0.0|
|Iris-setosa|[4.4,3.0,1.3,0.2]|       0.0|
|Iris-setosa|[4.4,3.2,1.3,0.2]|       0.0|
|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|Iris-setosa|[4.6,3.4,1.4,0.3]|       0.0|
+-----------+-----------------+----------+
only showing top 5 rows



## ***Definindo o Algoritmo***

Para o algoritmo de Árvore de Decisão, será utilizado o modulo do Pyspark DecisionTreeClassifier, onde é definido um modelo e aplicado um treinamento no mesmo.
E podemos ir um pouco além e analisar nosso modelo estatisticamente importando o modulo do Pyspark MulticlassClassificationEvaluator, permitindo avaliar a precisão do modelo de árvore de decisão.

In [82]:
from pyspark.ml.classification import DecisionTreeClassifier  #Biblioteca para o algoritmo da árvore de decisão
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  #Utilizada para encontrar as métricas de desempenho

In [83]:
modeloArvore = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="features")  #Definindo o modelo
model = modeloArvore.fit(train)  #Aplicando o treinamento

In [84]:
#Realizando a previsão
predictions = model.transform(test)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
|prediction|labelIndex|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 5 rows



Após a previsão, é possivel avaliar a precisão do modelo através dos seguintes comandos:

In [85]:
#Encontrando as métricas de avaliação para o modelo
avaliacao = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")

In [86]:
acuracia = avaliacao.evaluate(predictions)
print("Acurácia do Modelo =  ",(acuracia))

Acurácia do Modelo =   0.9056603773584906



# **Aplicação da Floresta Randômica**

Para a aplicação da floresta randomica, precisamos importar um módulos do Pyspark, o RandomForestClassifier. Com o RandomForestClassifier é possivel definir e analisar um modelo de floresta randomica.

In [21]:
from pyspark.ml.classification import RandomForestClassifier  #Classificador para o randomForest

In [22]:
modeloRF = RandomForestClassifier(labelCol="labelIndex",featuresCol="features", numTrees=10)  #Define o modelo
modelRF = modeloRF.fit(train)

In [23]:
#Realizando a previsão
predictions = modelRF.transform(test)
predictions.select("prediction", "labelIndex").show(5)

+----------+----------+
|prediction|labelIndex|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
+----------+----------+
only showing top 5 rows



In [60]:
#Encontrando as métricas de avaliação para o modelo
avaliacao = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction",metricName="accuracy")

In [25]:
acuracia = avaliacao.evaluate(predictions)
print("Acurácia do Modelo =  ",(acuracia))

Acurácia do Modelo =   0.9433962264150944


In [26]:
print(modelRF)

RandomForestClassificationModel (uid=RandomForestClassifier_d46e066f807c) with 10 trees
